In [10]:
# Importing Required Libraries
import psycopg2
from psycopg2 import sql
import pandas as pd

# Defining the execute_values Function
def execute_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        psycopg2.extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("The dataframe is inserted")
    cursor.close()

def create_table_from_csv(file_path, table_name):
    with open(file_path, 'r') as file:
        header = file.readline().strip()  # Read the first line as the header
    
    columns = header.split(',')
    column_definitions = [sql.Identifier(col).as_string(conn) + ' TEXT' for col in columns]  # Adjust the data type as needed
    
    create_table_query = sql.SQL("CREATE TABLE {} ({})").format(
        sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.SQL, column_definitions))
    )

    conn_string = psycopg2.connect(
        database="Postgres_Test",
        user='postgres',
        password='pocrapostgres',
        host='localhost',
        port='5433'
    )

    cursor = conn_string.cursor()
    cursor.execute(create_table_query)
    conn_string.commit()
    cursor.close()
    conn_string.close()

def import_csv_data(file_path, table_name, conn):
    with open(file_path, 'r') as file:
        cursor = conn.cursor()
        cursor.copy_expert(
            sql.SQL("COPY {} FROM STDIN WITH CSV HEADER").format(sql.Identifier(table_name)),
            file
        )
        conn.commit()
        cursor.close()

file_path = r'C:\Users\pocra\Desktop\PostgresSQL\CSV\MH_Villages_List.csv'

table_name = 'MH_Villages_List1'

# Creating the table
create_table_from_csv(file_path, table_name)

# Importing the CSV data
conn = psycopg2.connect(
    database="Postgres_Test",
    user='postgres',
    password='pocrapostgres',
    host='localhost',
    port='5433'
)
import_csv_data(file_path, table_name, conn)
conn.close()
